# Agent生命周期钩子（Hooks）

## Motivation
当多个Agent运行以及任务流转中，会有以下需求：
- 调试需求：获取Task交接给哪个Agent？Agent在调用工具之前，它的“思考”是什么？工具返回的结果又是什么？
- 监控和日志：我想记录下每个Agent的启动和结束时间，以及它处理了哪些任务，用于性能分析或审计。
- 扩展功能：我希望在Agent开始处理任务前，预先加载一些特定的数据；或者在Agent结束后，将结果保存到数据库。

它提供了一种非侵入式的方式，让你能够在Agent运行过程中的关键节点（比如开始、结束、调用工具、交接任务）“挂载”你自己的代码。
简而言之，相当于业务流程的中的数据埋点作用，用户业务执行中出现的转化率断点。去发现Agent在运行过程中的一些问题，并且可以在问题发生时及时通知开发者。

## Definition：Hooks的类型
### AgentHooks
- 定义：类；继承并且重写感兴趣的事件方法（如`on_start`, `on_tool_end`等）。然后，在创建具体的Agent实例时，通过`hooks=`参数将这个钩子类的实例挂载上去

- 作用范围：仅对挂载了该钩子的Agent实例生效。其他Agent实例不会受到影响
- 相当于给某个员工（Agent）安装了工作记录仪

### RunHooks
- 定义：类；继承并且重写感兴趣的事件方法（方法名略有不同，如`on_agent_start`, `on_agent_end`）。但是它是用在调用`Runner.run()`或`Runner.run_sync()`时，通过`hook=`参数传递进去的

- 作用范围：对本次运行涉及的所有Agent都生效。无论任务流经多少Agent，只要发生了对应的事件，这个钩子的方法就会触发
- 比喻：在整体项目（Run）的入口处安排一个“全局观察员”，记录所有员工（Agent）的行为

| 事件       | AgentHooks 方法名 | RunHooks 方法名  | 触发时机                     | 关键参数 (除了context, agent)                          |
|------------|-------------------|------------------|------------------------------|-------------------------------------------------------|
| Agent开始  | on_start          | on_agent_start   | Agent开始处理任务            | 无                                                    |
| Agent结束  | on_end            | on_agent_end     | Agent完成任务并产生输出      | output (Agent的最终输出)                              |
| 任务交接   | on_handoff        | on_handoff       | 任务从一个Agent交接给另一个  | source (源Agent) (AgentHooks) / from_agent, to_agent (RunHooks) |
| 工具使用开始| on_tool_start     | on_tool_start    | Agent准备调用一个工具        | tool (被调用的工具对象)                               |
| 工具使用结束| on_tool_end       | on_tool_end      | Agent成功调用工具并获得结果  | tool (被调用的工具对象), output/result (工具返回的结果字符串) |

## Design & Implementation：解析教程的代码

### AgentHooks
示例：监控Agent的开始与结束
```python
# ... (LLM配置等代码)

# 1. 定义继承自 AgentHooks 的类
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0 # 用于计数事件发生顺序

    # 2. 重写 on_start 方法
    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started")

    # 3. 重写 on_end 方法
    async def on_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} ended with output {output}")

# 4. 创建Agent时，将钩子实例传入 hooks 参数
agent = Agent(name="旅行助手", model=llm, hooks=MyAgentHooks(), instructions="You are a helpful assistant")

# ... (运行Agent的代码)
```

### RunHooks
示例：全局监控Agent的开始与结束
```python
# ... (LLM配置等代码)

# 1. 定义继承自 RunHooks 的类
class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    # 2. 重写 on_agent_start 方法 (注意方法名变化)
    async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started")

    # 3. 重写 on_agent_end 方法 (注意方法名变化)
    async def on_agent_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} ended with output {output}")

# 4. 创建Agent时，不传入hooks
agent = Agent(name="旅行助手", model=llm, instructions="You are a helpful assistant")

async def main():
    # 5. 调用Runner.run时，将钩子实例传入 hooks 参数
    result = await Runner.run(agent, hooks=MyRunHooks(), input="孟子全名叫什么?")
    print(result.final_output)

# ... (运行代码)
```
